In [1]:
import pandas as pd
import numpy as np
import math

SOLVER_MILO = "highs"
SOLVER_MINLO = "ipopt"

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["coin", "highs"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

AMPL Development Version 20240404 (MSVC 19.38.33135.0, 64-bit)
Demo license with maintenance expiring 20260131.
Using license file "c:\Users\thuduong\Anaconda3\envs\optima\Lib\site-packages\ampl_module_base\bin\ampl.lic".



In [2]:
## PARAMETER
BOUND = 0.3 # over cut - maxi bound on fc quantity
MIN_MARGIN = 8

# DATA TEST 1
### uu tien cat het 1 finished goods trong 1 cuon thep
### -> need cut chuyen thanh duong truoc khi sang finished goods khac
stocks = {
    "S1": {"width": 1219, "weight": 4395 },
    "S2": {"width": 1219, "weight": 9260},
    "S3": {"width": 1018, "weight": 3475},
    "S4": {"width": 1219, "weight": 8535},
    "S5": {"width": 236, "weight": 1571},
}

finish = {
    "F1": {"width": 235, "need_cut": 800 },
    "F2": {"width": 147, "need_cut": 1308},
    "F3": {"width": 136, "need_cut": 1290},
    "F4": {"width": 68, "need_cut": 600},
    "F5": {"width": 60, "need_cut": 170},
    "F6": {"width": 85, "need_cut": 132},
    "F7": {"width": 57, "need_cut": 100},
    "F8": {"width": 92, "need_cut": 100}, 
    "F9": {"width": 57, "need_cut": 735}, 
}


In [9]:
from codes.create_patterns import *

naive_patterns = make_patterns_by_weight_width(stocks, finish, BOUND, MIN_MARGIN)

# display(naive_patterns)
# ap_upper_bound(naive_patterns, finish)

# create pattern combination
patterns = []
for s in stocks:
    max_cuts_dict, min_cuts_dict = ap_stock_bound(naive_patterns,finish,s)
    generate_cut_combinations(s, min_cuts_dict, max_cuts_dict, patterns)

No feasible pattern was found for Stock S5 and FG F6
No feasible pattern was found for Stock S5 and FG F7
No feasible pattern was found for Stock S5 and FG F8


In [ ]:
def cut_patterns(stocks, finish, patterns):
    m = AMPL()
    # m.eval("reset data;")
    m.eval(
    """
        set S;
        set F;
        set P;

        # width stock
        param width_s{S};
        # width finished pieces
        param width_f{F};

        # how many f pieces are returned from pattern p
        param a{F, P};

        # which stock s is choosen for pattern p
        var b{S, P} binary;

        # Find the patterns of stock that minimize the loss
        minimize trim_loss {p in P}:
          sum{s in S} b[s, p] * width_s[s] - sum{f in F} a[f,p] * width_f[f];
        
        subject to assign_each_stock_to_pattern {p in P}:
          sum{s in S} b[s, p]  = 1;
        
        subject to feasible_pattern_max_margin {p in P}:
          sum{f in F} a[f,p] * width_f[f] >= 0.96 * sum{s in S} b[s,p] * width_s[s];
        
        subject to feasible_pattern_min_margin {p in P}:
          sum{f in F} a[f,p] * width_f[f] <= sum{s in S} b[s,p] * width_s[s] - 8;

    """
    )

    m.set["S"] = list(stocks.keys())
    m.set["F"] = list(finish.keys())
    m.set["P"] = list(range(len(patterns)))

    m.param["width_s"] = {s: stocks[s]["width"] for s in stocks.keys()}
    m.param["width_f"] = {f: finish[f]["width"] for f in finish.keys()}

    a = {
        (f, p): patterns[p]["cuts"][f]
        for p in range(len(patterns))
        for f in finish.keys()
    }
    m.param["a"] = a
    
    m.option["solver"] = SOLVER_MILO
    m.get_output("solve;")

    opt_patterns = []
    for s in stocks.keys():
        opt_patterns.append(
            {
                "stocks":[(s,p) for p in range(len(patterns)) if m.var["b"][s, p].value() > 0],
            }
        )

    return opt_patterns

In [ ]:
def cut_patterns_by_multi_stocks(stocks, finish, patterns):
    m = AMPL()
    m.eval("reset data;")
    m.eval(
    """
        set S;
        set F;
        set P;

        # width stock
        param width_s{S};
        # width finished pieces
        param width_f{F};
        
        # weight per unit of stock
        param wu{S};

        # upper bound with over-cut
        param f_upper_demand{F};
        param demand_finish{F};
        # param a_upper_bound{F};

        # how many f pieces are returned from pattern p
        param a{F, P};
        # var a{f in F, p in P} integer >= 0 <= a_upper_bound[f];
        # which stock s is choosen for pattern p
        var b{S, P} binary;

        # Find the patterns of stock that minimize the loss
        minimize trim_loss {p in P}:
          sum{s in S} b[s, p] * width_s[s] - sum{f in F} a[f,p] * width_f[f];
        
        subject to assign_each_stock_to_pattern {p in P}:
          sum{s in S} b[s, p]  = 1;
        
        subject to feasible_pattern_max_margin {p in P}:
          sum{f in F} a[f,p] * width_f[f] >= 0.96 * sum{s in S} b[s,p] * width_s[s];
        
        subject to feasible_pattern_min_margin {p in P}:
          sum{f in F} a[f,p] * width_f[f] <= sum{s in S} b[s,p] * width_s[s] - 8;

        subject to weight_demand {p in P, f in F}:
          a[f,p] * width_f[f] * sum{s in S} b[s, p] * wu[s] <= f_upper_demand[f];


    """
    )

    m.set["S"] = list(stocks.keys())
    m.set["F"] = list(finish.keys())
    m.set["P"] = list(range(len(patterns)))

    m.param["width_s"] = {s: stocks[s]["width"] for s in stocks.keys()}
    m.param["width_f"] = {f: finish[f]["width"] for f in finish.keys()}

    m.param["wu"] = {s: stocks[s]["weight"]/stocks[s]["width"] for s in list(stocks.keys())} # stock weight per unit (unique)
    m.param["demand_finish"] = {f: finish[f]["need_cut"] for f in finish.keys()}
    m.param["f_upper_demand"] = {f: finish[f]["need_cut"] + BOUND * finish[f]["need_cut"] for f in finish.keys()}

    a = {
        (f, p): patterns[p]["cuts"][f]
        for p in range(len(patterns))
        for f in finish.keys()
    }
    m.param["a"] = a
    
    m.option["solver"] = SOLVER_MILO
    m.get_output("solve;")

    opt_patterns = []
    for s in stocks.keys():
        opt_patterns.append(
            {
                "stocks":[(s,p) for p in range(len(patterns)) if m.var["b"][s, p].value() > 0],
            }
        )

    return opt_patterns

In [ ]:
### Add weight constraints

def cut_weight_patterns(stocks, finish, patterns):
    m = AMPL()
    m.eval("reset data;")
    m.eval(
    """
        set S;
        set F;
        set P;

        # width stock
        param width_s{S};
        # width finished pieces
        param width_f{F};
        
        # weight per unit of stock
        param wu{S};

        # upper bound with over-cut
        param f_upper_demand{F};
        param demand_finish{F};
        # param a_upper_bound{F};

        # how many f pieces are returned from pattern p
        param a{F, P};
        # var a{f in F, p in P} integer >= 0 <= a_upper_bound[f];
        # which stock s is choosen for pattern p
        var b{S, P} binary;

        # Find the patterns of stock that minimize the loss
        minimize trim_loss {p in P}:
          sum{s in S} b[s, p] * width_s[s] - sum{f in F} a[f,p] * width_f[f];
        
        subject to assign_each_stock_to_pattern {p in P}:
          sum{s in S} b[s, p]  = 1;
        
        subject to feasible_pattern_max_margin {p in P}:
          sum{f in F} a[f,p] * width_f[f] >= 0.96 * sum{s in S} b[s,p] * width_s[s];
        
        subject to feasible_pattern_min_margin {p in P}:
          sum{f in F} a[f,p] * width_f[f] <= sum{s in S} b[s,p] * width_s[s] - 8;

        subject to weight_demand {p in P, f in F}:
          a[f,p] * width_f[f] * sum{s in S} b[s, p] * wu[s] <= f_upper_demand[f];


    """
    )

    m.set["S"] = list(stocks.keys())
    m.set["F"] = list(finish.keys())
    m.set["P"] = list(range(len(patterns)))

    m.param["width_s"] = {s: stocks[s]["width"] for s in stocks.keys()}
    m.param["width_f"] = {f: finish[f]["width"] for f in finish.keys()}

    m.param["wu"] = {s: stocks[s]["weight"]/stocks[s]["width"] for s in list(stocks.keys())} # stock weight per unit (unique)
    m.param["demand_finish"] = {f: finish[f]["need_cut"] for f in finish.keys()}
    m.param["f_upper_demand"] = {f: finish[f]["need_cut"] + BOUND * finish[f]["need_cut"] for f in finish.keys()}

    a = {
        (f, p): patterns[p]["cuts"][f]
        for p in range(len(patterns))
        for f in finish.keys()
    }
    m.param["a"] = a
    
    m.option["solver"] = SOLVER_MILO
    m.get_output("solve;")

    opt_patterns = []
    for s in stocks.keys():
        opt_patterns.append(
            {
                "stocks":[(s,p) for p in range(len(patterns)) if m.var["b"][s, p].value() > 0],
            }
        )

    return opt_patterns

### Optimize on selected stock then compare the result

In [19]:
def cut_patterns_by_stock(stocks,s, finish, patterns):
    m = AMPL()
    m.eval("reset data;")
    m.eval(
    """
        set F;
        set P;

        # width stock
        param width_s integer;
        # weight per unit of stock
        param wu > 0;
        # width finished pieces
        param width_f{F};

        # upper bound with over-cut
        param f_upper_demand{F};
        param demand_finish{F};
        # param a_upper_bound{F};

        # how many f pieces are returned from pattern p
        param a{F, P};
        # var a{f in F, p in P} integer >= 0 <= a_upper_bound[f];
        # which stock s is choosen for pattern p
        var b{P} binary;

        # Find the patterns of stock that minimize the loss
        minimize trim_loss:
          width_s - sum{p in P, f in F} b[p] * a[f,p] * width_f[f];
        
        subject to assign_each_finish_to_pattern:
          sum{p in P} b[p]  = 1;
        
        subject to feasible_pattern_max_margin:
          sum{p in P, f in F} b[p]* a[f,p] * width_f[f] >= 0.96 * width_s;
        
        subject to feasible_pattern_min_margin:
          sum{p in P, f in F}  b[p] * a[f,p] * width_f[f] <= width_s - 8;

        subject to weight_demand {f in F}:
          sum{p in P} b[p] * a[f,p] * width_f[f] * wu <= f_upper_demand[f];
    """
    )
    m.set["F"] = list(finish.keys())
    m.set["P"] = list(range(len(patterns)))

    m.param["width_s"] = stocks[s]["width"] 
    m.param["width_f"] = {f: finish[f]["width"] for f in finish.keys()}

    m.param["wu"] = stocks[s]["weight"]/stocks[s]["width"]  # stock weight per unit (unique)
    m.param["demand_finish"] = {f: finish[f]["need_cut"] for f in finish.keys()}
    m.param["f_upper_demand"] = {f: finish[f]["need_cut"] + BOUND * finish[f]["need_cut"] for f in finish.keys()}

    a = {
        (f, p): patterns[p]["cuts"][f]
        for p in range(len(patterns))
        for f in finish.keys()
    }
    m.param["a"] = a
    
    m.option["solver"] = SOLVER_MILO
    m.get_output("solve;")

    opt_patterns = []
    opt_patterns.append(
        {
            "pattern":[p for p in range(len(patterns)) if m.var["b"][p].value() > 0],
            "stocks": s,
            "trim loss": [0]
        }
    )

    return opt_patterns

In [29]:
s = "S1"
filtered_patterns = [item for item in patterns if item['stock'] == s]

# len(filtered_patterns)
cut_patterns_by_stock(stocks,s, finish, filtered_patterns)

[{'pattern': [1142], 'stocks': 'S1', 'trim loss': [0]}]

In [26]:
s = "S3"
filtered_patterns = [item for item in patterns if item['stock'] == s]

# len(filtered_patterns)
cut_patterns_by_stock(stocks,s, finish, filtered_patterns)

1279

In [28]:
filtered_patterns[378]

{'stock': 'S3',
 'cuts': {'F1': 0,
  'F2': 2,
  'F8': 0,
  'F5': 0,
  'F9': 3,
  'F7': 0,
  'F4': 2,
  'F6': 0,
  'F3': 3}}

In [30]:
s = "S4"
filtered_patterns = [item for item in patterns if item['stock'] == s]

# len(filtered_patterns)
cut_patterns_by_stock(stocks,s, finish, filtered_patterns)

[{'pattern': [], 'stocks': 'S4', 'trim loss': [0]}]

In [31]:
s = "S5"
filtered_patterns = [item for item in patterns if item['stock'] == s]

# len(filtered_patterns)
cut_patterns_by_stock(stocks,s, finish, filtered_patterns)

[{'pattern': [], 'stocks': 'S5', 'trim loss': [0]}]